In [ ]:
import numpy as np
import pandas as pd
import requests
import gspread
gc = gspread.oauth()

# UKBBL Season Changeover Automation

This workflow is designed to simplify(?) the process of dealing with promotions / demotions of teams each season for the UKBBL league. It pulls league information from the goblinspy website, (semi)automatically processes this based on rankings and coaches decisions for next season, and then pushes the new division lists to googlesheets.

Inputs required:
1. the season settings below will need updating each season (principally 'division_dict', to reflect the unique division ID numbers)
2. a file called 'google_folder_id.txt', containing a single string for the folder ID number (this is the string after: https://drive.google.com/drive/folders/)

Preparation process:
1. download league tables
2. push these to a defined googlesheet
3. edit the googlesheet if required (will need to write code to download sheet again if this is required) 
4. create coach list, and push to defined googlesheet
5. manually add coach intentions to the googlesheet

Season changeover process:
1. download league tables (as above, but should download from googlesheet instead)
2. download coach list (with intentions for new season) from googlesheet
3. run changeover code (allocating new division levels, but not specific divisions)
4. push new listings to googlesheet
5. manually allocate coaches within each division level to their new divisions

In [ ]:
original_columns = ['coach_name','team_name','idcompetition','idrace','cas','td','ranking','points','sorting']

source_directory = 'season41_test_tables'


gspy_api_post_string = 'https://www.mordrek.com:666/api/comp/{}/sorting/{}'
gspy_api_get_string = 'https://www.mordrek.com:666/api/comp/{}/standings'

sorting_base_string = '{}%2C%20{}%2C%20{}%2C%20{}'

column_drop_list = ['idrace', 'idstanding', 
       'wins', 'draws', 'losses', 'td', 'td_opp', 'td_diff', 'cas', 'cas_opp',
       'cas_diff', 'concedes', 'team_value', 'kills',  'gp',
       'idteam', 'logo', 'twitch', 'youtube', 'idcoach']
column_keep_list = ['active', 'ranking', 'position',  'points', 'sorting', 'coach_name', 'team_name', 'active']

race_dict = {'1':'Human','2':'Dwarf','3':'Skaven','4':'Orc','5':'Lizard','6':'Goblin',
             '7':'Wood Elf','8':'Chaos','9':'Dark Elf','10':'Undead','11':'Halfling','12':'Norse',
             '13':'Amazon','14':'Eleven Union','15':'High Elf','16':'Khemri','17':'Necromantic','18':'Nurgle',
             '19':'Ogre','20':'Vampire','21':'Chaos Dwarf','22':'Underworld','23':'','24':'Brettonian',
             '25':'Kislev'}

with open('google_folder_id.txt', 'r') as file:
    google_folder_string = file.read().replace('\n', '')


In [ ]:
def download_table(division_string,sort_string,division_values):
    
    try:
        response = requests.post(gspy_api_post_string.format(division_string,sort_string))
        response = requests.get(gspy_api_get_string.format(division_string))
    except:
        print('wtf!')
    
    division_table = pd.DataFrame(response.json()['rows'],columns=response.json()['cols'])
    division_table['division_name'] = division_values[0]
    division_table['division_level'] = division_values[1]
    
    return(division_table)


In [ ]:
division_dict = {'12981':['Premiership West S41',1],
                     '12980':['Premiership East S41',1],
                     '12979':['Altdorf East S41',2],
                     '12978':['Altdorf West S41',2],
                     '12976':['Barren Hills West S41',3],
                     '12977':['Kolsa Hills East S41',3],
                     '12974':['Laurelorn Forest West S41',4],
                     '12975':['Gryphons Wood East S41',4]}

sorting_options = ['3-1-0','TDD','H2H','CW']

sort_string = sorting_base_string.format(sorting_options[0],sorting_options[1],sorting_options[2],sorting_options[3])


## Download division information from goblinspy

Run through the division list, and pull each down using the 'download_table' function. Then add the race name based on the raceid number, and convert the position and ranking values to numbers rather than strings.

In [ ]:
full_division_tables = pd.DataFrame()
for division_string,division_values in division_dict.items():
    full_division_tables = full_division_tables.append(download_table(division_string,sort_string,division_values))    


In [ ]:
division_tables

In [ ]:
division_tables = full_division_tables.copy(deep=True)
division_tables['race'] = [race_dict[x] for x in division_tables['idrace']]
division_tables.drop(columns=column_drop_list,inplace=True)


In [ ]:
division_tables['position']=[int(x) for x in division_tables['position']]
division_tables['points']=[int(x) for x in division_tables['points']]
division_tables['ranking']=[float(x) for x in division_tables['ranking']]
division_tables['sorting']=[float(x) for x in division_tables['sorting']]

## Uploading Divisions to a storage google sheet

Run these steps if the division tables need updating

In [ ]:
division_list_name = 'S41_division_tables'

In [ ]:
division_sheet = gc.create(title=division_list_name,folder_id=google_folder_string)
worksheet = division_sheet.sheet1
worksheet.update([division_tables.columns.values.tolist()] + division_tables.values.tolist())

In [ ]:
for division_string,division_values in division_dict.items():
    div_title = division_values[0]
    division_sheet.add_worksheet(div_title,rows=30,cols=30)
    div_temp = division_tables[division_tables['division_name']==div_title]
    division_sheet.worksheet(div_title).update([div_temp.columns.values.tolist()] + div_temp.values.tolist())

## Creating the coach list

Run these steps only once, to create a spreadsheet with division and coach names. Then fill the decisions
column in by hand.

In [ ]:
coach_list_name = 'coach_decisions_S41'

In [ ]:
coach_list = division_tables[['division_name','coach_name']]

In [ ]:
coach_sheet = gc.create(title=coach_list_name,folder_id=google_folder_string)
worksheet = coach_sheet.sheet1
worksheet.update([coach_list.columns.values.tolist()] + coach_list.values.tolist())

In [ ]:
worksheet.update('C1','decisions')
worksheet.format('A1:C1', {'textFormat': {'bold': True}})

## Download coach list, including decisions for next season

This pulls the coach decisions from the 

In [ ]:
coach_sheet = gc.open(title=coach_list_name)
worksheet = coach_sheet.sheet1

In [ ]:
dataframe = pd.DataFrame(worksheet.get_all_records())

In [ ]:
division_tables = division_tables.merge(dataframe[['coach_name','decisions']],how='left',on='coach_name')

## Preparing database for processing work

We will rename the division name column, and add some new columns.

Also the functions for assigning teams to the new division levels, removing dropouts and assigning rerollers to level 4, and promoting to fill gaps in upper levels will be defined.

In [ ]:
division_tables.columns

In [ ]:
division_tables = division_tables.rename(columns={'division_name':'orig_div_name','division_level':'orig_div_level'})

In [ ]:
division_tables['new_div_name'] = ''
division_tables['new_div_level'] = np.nan

In [ ]:
division_tables['position']=[int(x) for x in division_tables['position']]

In [ ]:
def assign_new_div_level(div_table,orig_level,promote_number,demote_number,division_number):
    all_index = div_table.loc[div_table['orig_div_level']==orig_level].index
    promote_index = div_table.loc[div_table['orig_div_level']==orig_level].loc[div_table['position']<=promote_number].index
    demote_index = div_table.loc[div_table['orig_div_level']==orig_level] \
                            .loc[div_table['position']>(division_number-demote_number)].index
    div_table.loc[all_index,'new_div_level'] = orig_level
    if promote_number > 0:
        div_table.loc[promote_index,'new_div_level'] = orig_level - 1
    if demote_number > 0:
        div_table.loc[demote_index,'new_div_level'] = orig_level + 1
    #return(div_table)

def assign_rerollers_remove_dropouts(div_table,reroll_string,dropout_string):
    reroll_index = div_table.loc[div_table['decisions']==reroll_string].index
    dropout_index = div_table.loc[div_table['decisions']==dropout_string].index
    div_table.loc[reroll_index,'new_div_level'] = 4
    div_table.loc[dropout_index,'new_div_level'] = -999
    #return(div_table)

def promote_to_fill_gaps(div_table,div_level,req_number_teams):
    supply_level = div_level+1.0
    starting_number_teams = div_table.loc[div_table['new_div_level']==div_level,'race'].count()
    for value in range(starting_number_teams,req_number_teams,1):
        search_value = div_table.loc[div_table['orig_div_level']==supply_level]\
                                .loc[div_table['new_div_level']==supply_level,'ranking'].max()
        team_index = div_table.loc[div_table['orig_div_level']==supply_level]\
                              .loc[div_table['new_div_level']==supply_level]\
                              .loc[div_table['ranking']==search_value].index
        div_table.loc[team_index[0],'new_div_level'] = div_level


## Assigning Teams for the New Season

Here we run the new season process.
1. dealing with promotions / demotions (carried out for each old division level, assigning how many will be promoted / demoted, as well as old division size)
2. assigning rerollers to division level 4, and assigning leaving coaches to level -999
3. promoting teams to fill gaps in each division level (starting from level 1, and working downwards)

Step 3 is run on a copy of the division tables, so that we can recover the original listing if anything goes wrong there.

After these steps we print out the numbers for each new division level, as a reality check, and for any teams which remained in their original division we allocate them the new division name now.

In [ ]:
assign_new_div_level(division_tables,1,0,3,12)
assign_new_div_level(division_tables,2,3,3,12)
assign_new_div_level(division_tables,3,3,3,12)
assign_new_div_level(division_tables,4,3,0,14)

In [ ]:
assign_rerollers_remove_dropouts(division_tables,'twist','out')

In [ ]:
div_test = division_tables.copy(deep=True)
promote_to_fill_gaps(div_test,1,24)
promote_to_fill_gaps(div_test,2,24)
promote_to_fill_gaps(div_test,3,24)

In [ ]:
div_test[['new_div_level','team_name']].groupby('new_div_level').count()

In [ ]:
div_test.loc[div_test['new_div_level']==div_test['orig_div_level'],'new_div_name'] = \
                      [x.replace('S41','S42') for x in div_test.loc[div_test['new_div_level']==div_test['orig_div_level'],'orig_div_name'] ]

In [ ]:
div_test.loc[div_test['decisions']=='twist',['team_name','new_div_name','race',]]=''

In [ ]:
div_test.loc[div_test['new_div_level']==1,['team_name','coach_name','new_div_name','race','orig_div_level','decisions']]

## Uploading new division tables to googlesheets

Do this once the messy sorting has been done. The final tidying up of the division allocations can be carried out by hand in the googlesheets document.

In [ ]:
new_division_list_name = 'S42_division_tables'

In [ ]:
division_sheet = gc.create(title=new_division_list_name,folder_id=google_folder_string)
#worksheet = division_sheet.sheet1
#worksheet.update([division_tables.columns.values.tolist()] + division_tables.values.tolist())

In [ ]:
for level in div_test['new_div_level'].unique():
    sheet_title = 'level {}'.format(level)
    division_sheet.add_worksheet(sheet_title,rows=40,cols=20)
    div_temp = div_test.loc[div_test['new_div_level']==level,['team_name','coach_name','new_div_name','race','orig_div_level','decisions']]
    division_sheet.worksheet(sheet_title).update([div_temp.columns.values.tolist()] + div_temp.values.tolist())

In [ ]:
div_test['new_div_level'].unique()